### Set up environment

In [1]:
from src.get_data import get_df

### Run the Query

In [2]:
# Write the SQL query

## Morning lab values
# From MIMIC-IV database in the table `physionet-data.mimiciv_hosp.labevents`
# The lab values are selected from the following itemids:
# 50912, 51265, 51222, 51301, 51006, 50983, 50882, 50971
# The lab values are selected from the following hours: 5 - 8 am

query = """
WITH cohort AS (
  SELECT
    hadm_id,
    subject_id,
    itemid,
    charttime,
    EXTRACT(HOUR FROM charttime) AS charthour,
    storetime,
    EXTRACT(HOUR FROM storetime) AS storehour,
    EXTRACT(DATE FROM charttime) AS chartday,
    valuenum
  FROM
    `physionet-data.mimiciv_hosp.labevents`
  WHERE EXTRACT(HOUR FROM charttime) BETWEEN 5 AND 8
    AND itemid IN (50912, 51265, 51222, 51301, 51006, 50983, 50882, 50971)
),
patdailycnt AS (
  SELECT
    subject_id,
    chartday,
    COUNT(DISTINCT itemid) AS cnt
  FROM cohort where SAFE_CAST(valuenum AS FLOAT64) IS NOT NULL AND valuenum IS NOT NULL
  GROUP BY subject_id, chartday
)
  SELECT cohort.*,patdailycnt.cnt
  FROM cohort
  RIGHT JOIN patdailycnt
  ON cohort.subject_id = patdailycnt.subject_id AND cohort.chartday = patdailycnt.chartday
  WHERE patdailycnt.cnt = 8

"""

In [ ]:
df = get_df(query, save_csv=True, csv_name="morning_lab_values.csv")